In [1]:
# data access
import earthaccess

# custom tool functions
import importlib
import tools
import makeL3_scripts
importlib.reload(tools)
importlib.reload(makeL3_scripts)
from tools import *
from makeL3_scripts import makeL3

# data manipulation
import numpy as np
import xarray as xr
import cf_xarray  # noqa: F401
import rasterio
import rioxarray as rio
from rasterio.enums import Resampling

# plotting
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
auth = earthaccess.login(persist=True)

In [3]:
tspan = ("2025-07-01", "2025-08-01")
sep_bbox = (-110, 10, -70, 35) # SEP, WSEN
so_bbox = (-170,-60,-120,-50) # SO, WSEN

#####

bbox = sep_bbox
loc = "SEP"
res = 1.0
strres = {1.0:"1deg", 0.1:"0p1deg"}[res]

In [4]:
results = earthaccess.search_datasets(instrument="oci")
for item in results:
    summary = item.summary()
    # print(summary["short-name"])

In [5]:
results_harp = earthaccess.search_data(
    short_name="PACE_HARP2_L2_CLOUD_GPC_NRT",
    temporal=tspan,
    bounding_box=bbox,
)
len(results_harp)

184

In [6]:
results_harp = earthaccess.search_data(
    short_name="PACE_HARP2_L2_CLOUD_GPC_NRT",
    temporal=tspan,
    bounding_box=bbox,
)
len(results_harp)

184

In [7]:
results_oci = earthaccess.search_data(
    short_name="PACE_OCI_L2_CLOUD_NRT",
    temporal=tspan,
    bounding_box=bbox,
)
len(results_oci)

180

In [8]:
begintime_harp = [i["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"] for i in results_harp]
begintime_oci = [i["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"] for i in results_oci]
begintime_both = list(set(begintime_harp) & set(begintime_oci))

In [9]:
clean_results_harp = []
begintime_harp = []
for i in results_harp:
    begin_harp = i["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"]
    if begin_harp in begintime_both:
        clean_results_harp.append(i)
        begintime_harp.append(begin_harp)

idx = np.argsort(begintime_harp)
sorted_results_harp = list(np.array(clean_results_harp)[idx])

In [10]:
clean_results_oci = []
begintime_oci = []
for i in results_oci:
    begin_oci = i["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"]
    if begin_oci in begintime_both:
        clean_results_oci.append(i)
        begintime_oci.append(begin_oci)

idx = np.argsort(begintime_oci)
sorted_results_oci = list(np.array(clean_results_oci)[idx])

In [11]:
paths_harp = earthaccess.open(sorted_results_harp)

QUEUEING TASKS | :   0%|          | 0/178 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/178 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/178 [00:00<?, ?it/s]

In [12]:
paths_oci = earthaccess.open(sorted_results_oci)

QUEUEING TASKS | :   0%|          | 0/178 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/178 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/178 [00:00<?, ?it/s]

In [ ]:
dt_harp = xr.open_datatree(paths_harp[0])
dt_oci = xr.open_datatree(paths_oci[0])
ds_harp = xr.merge(dt_harp.to_dict().values())
ds_oci = xr.merge(dt_oci.to_dict().values())
ds = xr.merge([ds_harp, ds_oci])
ds = ds.set_coords(("longitude", "latitude"))
ds_oci